# __WSI - ćwiczenie 5.__

### __Sztuczne sieci neuronowe__

#### __Treść ćwiczenia__

- Celem cwiczenia jest implementacja perceptronu wielowarstwowego oraz wybranego algorytmu
optymalizacji gradientowej z algorytmem propagacji wstecznej.
- Nastepnie nalezy wytrenowac perceptron wielowarstwowy do klasyfikacji zbioru danych wine
(https://archive.ics.uci.edu/ml/datasets/wine). Zbiór ten dostepny jest w pakiecie scikitlearn
(sklearn.datasets.load wine).

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, auc, RocCurveDisplay, PrecisionRecallDisplay, recall_score, precision_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from seaborn import heatmap
import plotly.express as px
from math import log, inf
from sklearn.utils import resample, shuffle

RNG = np.random.default_rng()

cele: 
- perceptron wielowarstwowy, implementacja ze zmienną ilością warstw głębokich oraz zmienną ilością długości wektora neuronów
- kilka algorytmów optymalizacji wag sieci (gradient prosty, SGD, algorytm ewolucyjny??)

zadania:
1. model sieci
2. propagacja wsteczna
3. optymalizacja wag

weight matrix per layer:

$$
\theta^{l}=
\left[\begin{array}{ccc}
\omega_{11}& \cdots&\omega_{1k}\\
\vdots&\ddots&\vdots\\
\omega_{n1}&\cdots&\omega_{nk}
\end{array}\right]
$$
where  $ \omega_{ij} $ is the $j$-th weight of the $i$-th neuron (in layer $l$) 

weight matrix per layer:

$$
\Theta=
\left[\begin{array}{ccc}
\omega_{11}& \cdots&\omega_{1k}\\
\vdots&\ddots&\vdots\\
\omega_{n1}&\cdots&\omega_{nk}
\end{array}\right]
$$
where  $ \omega_{ij} $ is the $j$-th weight of the $i$-th neuron (in layer $l$) 